# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-3d7c3e393e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Economy of EU States") 

Consider the following exploratory information need:

> Compare businesses across different sectors and types in E.U. countries

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wd:Q458`      | E.U.         | node |
| `wd:Q142`      | France       | node      |
| `wd:Q4830453`  | Business     | node      |
| `wd:Q6881511`  | Enterprise   | node      |
| `wd:Q29110228` | AXA          | node |
| `wd:Q43183`    | insurance    | node |




Also consider

```
{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}



```

is the BGP to retrieve all **french enterprises and businesses**

## Workload Goals

1. Identify the BGP for obtaining the type, legal form, and industry of a company

2. Identify the BGP to retrieve all companies owned by a company located in a EU country

3. Which company has the largest presence in E.U.?

4. Companies have different 'legal forms', compare the number of companies divided in different legal forms

5. Analyze the number of companies per type, legal form, and industry in each state
 
   5.1 What are the top-3 legal form in E.U.? 
   
   5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each country for a given legal form?
   
   5.3 Which business in each country owns more businesses in other E.U. countries?
   
   5.4 What can we say about industry sectors in various countries?


In [1]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

{ 
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '4577')]


1

In [ ]:
#Identify the BGP for obtaining the type, legal form, and industry of a company

In [30]:
#here i get the relationships for a company

queryString = """
SELECT ?p ?business ?relation ?relationName
WHERE { 

{ 
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}
?p ?relation ?g.
?p <http://schema.org/name> ?business.
?relation <http://schema.org/name> ?relationName.
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('relation', 'http://www.wikidata.org/prop/direct/P1454'), ('relationName', 'legal form')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('relation', 'http://www.wikidata.org/prop/direct/P154'), ('relationName', 'logo image')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('relation', 'http://www.wikidata.org/prop/direct/P159'), ('relationName', 'headquarters location')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('relation', 'http://www.wikidata.org/prop/direct/P17'), ('relationName', 'country')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('relation', 'http://www.wikidata.org/prop/direct/P2002'), ('relationName', 'Twitter username')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('relation', 'http://www.wikidata.org/prop/direct/P2003'), ('relationNa

200

In [ ]:
#in the previous query i found the relationship total asset which i will use later P2403
#the BGP for legal form found in the query above is P1454, the one for industry is P452, the one for type is wdt:P31 which is the one for instance of

In [11]:
#here i find the relationship member of for the eu
queryString = """
SELECT ?country ?relation ?relationName
WHERE { 

 



wd:Q142 ?relation wd:Q458.

?relation <http://schema.org/name> ?relationName.
} 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('relation', 'http://www.wikidata.org/prop/direct/P463'), ('relationName', 'member of')]


1

In [ ]:
#i found the BGP P463 whic represent member of I have than to use this for consider country in the european union (look the last query to a response
#to the question 2)

#the BGP for legal form found in the query above is P1454, the one for industry is P452, the one for type is wdt:P31 which is the one for instance of

In [10]:
#in this query i return the type, legal form, and industry of a company every company not only in the E.U. 
queryString = """
SELECT ?p ?business ?legalForm ?industry ?type
WHERE { 

{ 
 
?p wdt:P31 wd:Q6881511  . 
} UNION {
 
?p wdt:P31 wd:Q4830453  . 
}
?p  wdt:P1454 ?legalForm.
?p wdt:P452 ?industry.
?p wdt:P31 ?type.
?p <http://schema.org/name> ?business.

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('legalForm', 'http://www.wikidata.org/entity/Q1562410'), ('industry', 'http://www.wikidata.org/entity/Q55638'), ('type', 'http://www.wikidata.org/entity/Q19307174')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('legalForm', 'http://www.wikidata.org/entity/Q1562410'), ('industry', 'http://www.wikidata.org/entity/Q55638'), ('type', 'http://www.wikidata.org/entity/Q81989119')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('legalForm', 'http://www.wikidata.org/entity/Q1562410'), ('industry', 'http://www.wikidata.org/entity/Q55638'), ('type', 'http://www.wikidata.org/entity/Q1137109')]
[('p', 'http://www.wikidata.org/entity/Q5366830'), ('business', 'DMM.com'), ('legalForm', 'http://www.wikidata.org/entity/Q1562410'), ('industry', 'http://www.wikidata.org/entity/Q55638'), ('type', 'http://www.wikidata.org/entity/Q35127')]
[('p', 'http://www.wikid

50

In [21]:

#in this query i return the type, legal form, and industry of a company every company only in the E.U. 
queryString = """
SELECT ?p ?business ?legalForm ?industry ?type
WHERE { 

{ 
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country. 
 ?p wdt:P31 wd:Q6881511  . 
} UNION {
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country.
 ?p wdt:P31 wd:Q4830453  . 
}
?p  wdt:P1454 ?legalForm.
?p wdt:P452 ?industry.
?p wdt:P31 ?type.
?p <http://schema.org/name> ?business.

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1642179'), ('business', 'Allied Irish Banks'), ('legalForm', 'http://www.wikidata.org/entity/Q6832945'), ('industry', 'http://www.wikidata.org/entity/Q29584334'), ('type', 'http://www.wikidata.org/entity/Q15911314')]
[('p', 'http://www.wikidata.org/entity/Q1642179'), ('business', 'Allied Irish Banks'), ('legalForm', 'http://www.wikidata.org/entity/Q6832945'), ('industry', 'http://www.wikidata.org/entity/Q29584334'), ('type', 'http://www.wikidata.org/entity/Q43229')]
[('p', 'http://www.wikidata.org/entity/Q1642179'), ('business', 'Allied Irish Banks'), ('legalForm', 'http://www.wikidata.org/entity/Q6832945'), ('industry', 'http://www.wikidata.org/entity/Q29584334'), ('type', 'http://www.wikidata.org/entity/Q4830453')]
[('p', 'http://www.wikidata.org/entity/Q1642179'), ('business', 'Allied Irish Banks'), ('legalForm', 'http://www.wikidata.org/entity/Q6832945'), ('industry', 'http://www.wikidata.org/entity/Q29584334'), ('type', 'http://www.w

29

In [15]:
#in the first query i found the relationship owned by P127



In [14]:
#answer to question 2 
#Identify the BGP to retrieve all companies owned by a company located in a EU country
queryString = """
SELECT ?country AS ?countryOwner  ?p AS ?owner ?owned
{

{
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P1830 ?owned.
?owned wdt:P31 wd:Q4830453  . 





}LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('countryOwner', 'http://www.wikidata.org/entity/Q27'), ('owner', 'http://www.wikidata.org/entity/Q329494'), ('owned', 'http://www.wikidata.org/entity/Q7277940')]
[('countryOwner', 'http://www.wikidata.org/entity/Q35'), ('owner', 'http://www.wikidata.org/entity/Q27231611'), ('owned', 'http://www.wikidata.org/entity/Q27231607')]
[('countryOwner', 'http://www.wikidata.org/entity/Q34'), ('owner', 'http://www.wikidata.org/entity/Q505922'), ('owned', 'http://www.wikidata.org/entity/Q16992781')]
[('countryOwner', 'http://www.wikidata.org/entity/Q142'), ('owner', 'http://www.wikidata.org/entity/Q274591'), ('owned', 'http://www.wikidata.org/entity/Q3045865')]
[('countryOwner', 'http://www.wikidata.org/entity/Q34'), ('owner', 'http://www.wikidata.org/entity/Q505922'), ('owned', 'http://www.wikidata.org/entity/Q3087840')]
[('countryOwner', 'http://www.wikidata.org/entity/Q34'), ('owner', 'http://www.wikidata.org/entity/Q505922'), ('owned', 'http://www.wikidata.org/entity/Q4024778')]
[('

100

In [ ]:
 #3. Which company has the largest presence in E.U.?

In [16]:
#Rifacendomi alla query precedente decido di usare come criterio la compagnia che possiede più compagnie europee


queryString = """
SELECT COUNT(distinct ?p) AS ?owned ?owner
WHERE { 

{ 
?p wdt:P17 ?country.
?country wdt:P463 wd:Q458.
 ?p wdt:P31 wd:Q6881511  . 

} UNION {

 ?p wdt:P31 wd:Q4830453  .
?p wdt:P17 ?country.
?country wdt:P463 wd:Q458.
 
}
 


?p wdt:P127 ?owner.


}GROUP BY ?owner 
ORDER BY DESC (?owned)

LIMIT 5
"""

print("Results")
run_query(queryString)







Results
[('owned', '34'), ('owner', 'http://www.wikidata.org/entity/Q219635')]
[('owned', '29'), ('owner', 'http://www.wikidata.org/entity/Q183')]
[('owned', '23'), ('owner', 'http://www.wikidata.org/entity/Q292750')]
[('owned', '19'), ('owner', 'http://www.wikidata.org/entity/Q64')]
[('owned', '18'), ('owner', 'http://www.wikidata.org/entity/Q442637')]


5

In [ ]:
#so the company which owns more E.U. companies is BlackRock

In [126]:
#NON GUARDARE SBAGLIATA (VERSIONE PRECEDENTE DI RISPOSTA A QUERY 3)
queryString = """
SELECT ?employers ?p
WHERE { 

{ 
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country. 
 ?p wdt:P31 wd:Q6881511  . 
 ?p wdt:P1128 ?employers.
} 
UNION
{
 
 ?country wdt:P463 wd:Q458.
 ?p wdt:P27 ?country.
 ?p wdt:P31 wd:Q4830453  .
 ?p wdt:P1128 ?employers.
 
}

?p <http://schema.org/name> ?business.

} 
ORDER BY DESC (?employers)
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [28]:
#SBAGLIATA (VERSIONE PRECEDENTE DI RISPOSTA A QUERY 3)
#att rifare, cosa si intende per presence?
queryString = """
SELECT  ?p
WHERE { 

{ 
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country. 
 ?p wdt:P31 wd:Q6881511  . 
} UNION {
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country.
 ?p wdt:P31 wd:Q4830453  . 
}

?p <http://schema.org/name> ?business.

} GROUP BY ?p
ORDER BY DESC (?presence)
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('presence', '2'), ('p', 'http://www.wikidata.org/entity/Q77353644')]
[('presence', '2'), ('p', 'http://www.wikidata.org/entity/Q534903')]
[('presence', '2'), ('p', 'http://www.wikidata.org/entity/Q3546460')]
[('presence', '2'), ('p', 'http://www.wikidata.org/entity/Q1534581')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q99969037')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q99971803')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q99972397')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q96994625')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q96996529')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q99970063')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q99969957')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q99971645')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q99969954')]
[('presence', '1'), ('p', 'http://www.wikidata.org/entity/Q96996119')]
[(

50

In [22]:
#compare the number of companies divided in different legal forms (i consider companies in the E.U.)
 
queryString = """
SELECT COUNT(?p) ?legalForm 
WHERE { 

{ 
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country. 
 ?p wdt:P31 wd:Q6881511  . 
} UNION {
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country.
 ?p wdt:P31 wd:Q4830453  . 
}
?p  wdt:P1454 ?legalForm.



}GROUP BY (?legalForm) 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1'), ('legalForm', 'http://www.wikidata.org/entity/Q3742494')]
[('callret-0', '1'), ('legalForm', 'http://www.wikidata.org/entity/Q510785')]
[('callret-0', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q166280')]
[('callret-0', '3'), ('legalForm', 'http://www.wikidata.org/entity/Q1555574')]
[('callret-0', '1'), ('legalForm', 'http://www.wikidata.org/entity/Q6832945')]
[('callret-0', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q15091621')]
[('callret-0', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q891723')]
[('callret-0', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q422053')]
[('callret-0', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q134161')]
[('callret-0', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q1436139')]


10

In [23]:
#What are the top-3 legal form in E.U.? 
queryString = """
SELECT COUNT(?p) AS ?number ?legalForm 
WHERE { 

{ 


 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country. 
 ?p wdt:P31 wd:Q6881511  . 
} UNION {
 ?country wdt:P463 wd:Q458.

 ?p wdt:P27 ?country.
 ?p wdt:P31 wd:Q4830453  . 
}
?p  wdt:P1454 ?legalForm.



}GROUP BY (?legalForm) 
ORDER BY DESC (?number)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('number', '3'), ('legalForm', 'http://www.wikidata.org/entity/Q1555574')]
[('number', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q166280')]
[('number', '2'), ('legalForm', 'http://www.wikidata.org/entity/Q15091621')]


3

In [37]:

#For which companies is defined some form of income or market capitalization or total assets? 
#What is the min, max, and average in each country for a given legal form?
#in this query a return the companies with their total asset P2403 it is not specified companies inside the E.U. so i consider all companies
queryString = """
SELECT ?p ?business ?g
WHERE { 

{ 
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}
?p wdt:P2403 ?g.
?p <http://schema.org/name> ?business.

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q8183600'), ('business', 'Agrostroj Pelhřimov'), ('g', '7824063000')]
[('p', 'http://www.wikidata.org/entity/Q80788'), ('business', 'DNB Bank'), ('g', '2793294000000')]
[('p', 'http://www.wikidata.org/entity/Q334122'), ('business', 'Merrill Lynch'), ('g', '603000000000')]
[('p', 'http://www.wikidata.org/entity/Q341139'), ('business', 'Česká zbrojovka Strakonice'), ('g', '2165963000')]
[('p', 'http://www.wikidata.org/entity/Q4162216'), ('business', 'Energetický a průmyslový holding'), ('g', '13329000000')]
[('p', 'http://www.wikidata.org/entity/Q4162216'), ('business', 'Energetický a průmyslový holding'), ('g', '342889000000')]
[('p', 'http://www.wikidata.org/entity/Q18552286'), ('business', 'Zealand Pharma'), ('g', '694626000')]
[('p', 'http://www.wikidata.org/entity/Q41365932'), ('business', 'AGEL'), ('g', '10518247000')]
[('p', 'http://www.wikidata.org/entity/Q41567534'), ('business', 'PPF'), ('g', '2203115000')]
[('p', 'http://www.wikid

20

In [41]:
#What is the min, max, and average in each country for a given legal form?
#in this query a return the min, max, and average for each country and legal form it is not specified companies inside the E.U. so i consider all companies
#i can add ?country wdt:P463 wd:Q458. if I want to limit anly for E.U. countries
queryString = """
SELECT MAX(?g) AS ?max MIN(?g) AS ?min AVG(?g) AS ?avg ?country ?legalform
WHERE { 

{ 
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}
?p wdt:P2403 ?g.

?p  wdt:P1454 ?legalform.


}GROUP BY ?country ?legalform 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('max', '86933'), ('min', '86933'), ('avg', '86933'), ('country', 'http://www.wikidata.org/entity/Q35'), ('legalform', 'http://www.wikidata.org/entity/Q15649047')]
[('max', '53801000000'), ('min', '53801000000'), ('avg', '53801000000'), ('country', 'http://www.wikidata.org/entity/Q39'), ('legalform', 'http://www.wikidata.org/entity/Q17375963')]
[('max', '845000000000'), ('min', '158904000'), ('avg', '93504643916.666666666666667'), ('country', 'http://www.wikidata.org/entity/Q55'), ('legalform', 'http://www.wikidata.org/entity/Q1436139')]
[('max', '8467304000'), ('min', '8467304000'), ('avg', '8467304000'), ('country', 'http://www.wikidata.org/entity/Q398'), ('legalform', 'http://www.wikidata.org/entity/Q891723')]
[('max', '196766000'), ('min', '196766000'), ('avg', '196766000'), ('country', 'http://www.wikidata.org/entity/Q131964'), ('legalform', 'http://www.wikidata.org/entity/Q3742494')]
[('max', '111993000000'), ('min', '111993000000'), ('avg', '111993000000'), ('country', 

20

In [ ]:
#Which business in each country owns more businesses in other E.U. countries? I assume that both the businesses must be in an E.U. country

In [85]:
#Which business in each country owns more businesses in other E.U. countries?
#SBAGLIATA VAI AVANTI
queryString = """
SELECT ?country2 ?Number ?owner
{


{
SELECT ?country2   MAX(?NumOwned) AS ?Number
{
SELECT   ?country2 ?owner COUNT(?p) as ?NumOwned
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P127 ?owner.
?country2 wdt:P463 wd:Q458.
?owner wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?owner


}GROUP BY ?country2

}



{
SELECT   ?country2 ?owner COUNT(?p) as ?Number
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P127 ?owner.
?country2 wdt:P463 wd:Q458.
?owner wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?owner

}



}

LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('country2', 'http://www.wikidata.org/entity/Q33'), ('Number', '7'), ('owner', 'http://www.wikidata.org/entity/Q1418')]
[('country2', 'http://www.wikidata.org/entity/Q45'), ('Number', '4'), ('owner', 'http://www.wikidata.org/entity/Q1341222')]
[('country2', 'http://www.wikidata.org/entity/Q31'), ('Number', '4'), ('owner', 'http://www.wikidata.org/entity/Q941020')]
[('country2', 'http://www.wikidata.org/entity/Q41'), ('Number', '2'), ('owner', 'http://www.wikidata.org/entity/Q2643648')]
[('country2', 'http://www.wikidata.org/entity/Q213'), ('Number', '6'), ('owner', 'http://www.wikidata.org/entity/Q20120422')]
[('country2', 'http://www.wikidata.org/entity/Q142'), ('Number', '12'), ('owner', 'http://www.wikidata.org/entity/Q77846713')]
[('country2', 'http://www.wikidata.org/entity/Q218'), ('Number', '2'), ('owner', 'http://www.wikidata.org/entity/Q104640458')]
[('country2', 'http://www.wikidata.org/entity/Q215'), ('Number', '2'), ('owner', 'http://www.wikidata.org/entity/Q106509

30

In [120]:
#SBAGLIATA VAI ALLA PROSSIMA DA FARE
#Which business in each country owns more businesses in other E.U. countries?
queryString = """
SELECT ?country2 ?Number ?p
{

{

SELECT   ?country2 ?p COUNT(?owned) as ?Number
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P1830 ?owned.
?country2 wdt:P463 wd:Q458.
?owned wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?p

}




{
SELECT ?country2  MAX(?NumOwned) AS ?Number
{
SELECT   ?country2 ?p COUNT(?owned) as ?NumOwned
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P1830 ?owned.
?country2 wdt:P463 wd:Q458.
?owned wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?p


}GROUP BY ?country2 ?p 

}







}

LIMIT 30
"""

print("Results")
run_query(queryString)










Results
[('country2', 'http://www.wikidata.org/entity/Q183'), ('Number', '2'), ('p', 'http://www.wikidata.org/entity/Q670482')]
[('country2', 'http://www.wikidata.org/entity/Q36'), ('Number', '2'), ('p', 'http://www.wikidata.org/entity/Q19305480')]
[('country2', 'http://www.wikidata.org/entity/Q145'), ('Number', '2'), ('p', 'http://www.wikidata.org/entity/Q565050')]
[('country2', 'http://www.wikidata.org/entity/Q183'), ('Number', '2'), ('p', 'http://www.wikidata.org/entity/Q3737496')]
[('country2', 'http://www.wikidata.org/entity/Q34'), ('Number', '2'), ('p', 'http://www.wikidata.org/entity/Q664399')]
[('country2', 'http://www.wikidata.org/entity/Q145'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q160054')]
[('country2', 'http://www.wikidata.org/entity/Q183'), ('Number', '2'), ('p', 'http://www.wikidata.org/entity/Q27485599')]
[('country2', 'http://www.wikidata.org/entity/Q183'), ('Number', '2'), ('p', 'http://www.wikidata.org/entity/Q298824')]
[('country2', 'http://www.wik

30

In [7]:
#SBAGLIATA VAI ALLA PROSSIMA
#Which business in each country owns more businesses in other E.U. countries?
queryString = """
SELECT ?country2 MAX(?Number) 
{

SELECT ?country2  ?NumOwned AS ?Number ?p
{
SELECT   ?country2 ?p COUNT(distinct ?owned) as ?NumOwned
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P1830 ?owned.
?country2 wdt:P463 wd:Q458.
?owned wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?p


}


}

LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country2', 'http://www.wikidata.org/entity/Q36'), ('callret-1', '2')]
[('country2', 'http://www.wikidata.org/entity/Q34'), ('callret-1', '3')]
[('country2', 'http://www.wikidata.org/entity/Q28'), ('callret-1', '1')]
[('country2', 'http://www.wikidata.org/entity/Q40'), ('callret-1', '1')]
[('country2', 'http://www.wikidata.org/entity/Q183'), ('callret-1', '4')]
[('country2', 'http://www.wikidata.org/entity/Q38'), ('callret-1', '4')]
[('country2', 'http://www.wikidata.org/entity/Q224'), ('callret-1', '1')]
[('country2', 'http://www.wikidata.org/entity/Q145'), ('callret-1', '4')]
[('country2', 'http://www.wikidata.org/entity/Q211'), ('callret-1', '1')]
[('country2', 'http://www.wikidata.org/entity/Q218'), ('callret-1', '2')]
[('country2', 'http://www.wikidata.org/entity/Q233'), ('callret-1', '1')]
[('country2', 'http://www.wikidata.org/entity/Q32'), ('callret-1', '1')]
[('country2', 'http://www.wikidata.org/entity/Q213'), ('callret-1', '13')]
[('country2', 'http://www.wikidata.

26

In [6]:
#look the next one
#Which business in each country owns more businesses in other E.U. countries? i select only the maximum but not the business 
queryString = """
SELECT ?country2  MAX(?NumOwned) AS ?Number 
{
SELECT ?country2 ?p COUNT(distinct ?owned) as ?NumOwned
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P1830 ?owned.
?country2 wdt:P463 wd:Q458.
?owned wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?p


}GROUP BY ?country2



LIMIT 1000
"""

print("Results")
run_query(queryString)

Results
[('country2', 'http://www.wikidata.org/entity/Q36'), ('Number', '2')]
[('country2', 'http://www.wikidata.org/entity/Q34'), ('Number', '3')]
[('country2', 'http://www.wikidata.org/entity/Q28'), ('Number', '1')]
[('country2', 'http://www.wikidata.org/entity/Q40'), ('Number', '1')]
[('country2', 'http://www.wikidata.org/entity/Q183'), ('Number', '4')]
[('country2', 'http://www.wikidata.org/entity/Q38'), ('Number', '4')]
[('country2', 'http://www.wikidata.org/entity/Q224'), ('Number', '1')]
[('country2', 'http://www.wikidata.org/entity/Q145'), ('Number', '4')]
[('country2', 'http://www.wikidata.org/entity/Q211'), ('Number', '1')]
[('country2', 'http://www.wikidata.org/entity/Q218'), ('Number', '2')]
[('country2', 'http://www.wikidata.org/entity/Q233'), ('Number', '1')]
[('country2', 'http://www.wikidata.org/entity/Q32'), ('Number', '1')]
[('country2', 'http://www.wikidata.org/entity/Q213'), ('Number', '13')]
[('country2', 'http://www.wikidata.org/entity/Q142'), ('Number', '6')]
[('

26

In [3]:
#Which business in each country owns more businesses in other E.U. countries? if there are more than one with the same number i return both
queryString = """
SELECT ?country2 ?Number ?p
{

{
SELECT ?country2  MAX(?NumOwned) AS ?Number 
{
SELECT ?country2 ?p COUNT(distinct ?owned) as ?NumOwned
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P1830 ?owned.
?country2 wdt:P463 wd:Q458.
?owned wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?p


}GROUP BY ?country2
}


{


SELECT ?country2 ?p COUNT(distinct ?owned) as ?Number
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}

?p wdt:P1830 ?owned.
?country2 wdt:P463 wd:Q458.
?owned wdt:P17 ?country2  .
FILTER(?country2!=?country)
} GROUP BY ?country2 ?p

}

}ORDER BY DESC (?country2)

LIMIT 1000
"""

print("Results")
run_query(queryString)

Results
[('country2', 'http://www.wikidata.org/entity/Q45'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q122141')]
[('country2', 'http://www.wikidata.org/entity/Q45'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q842108')]
[('country2', 'http://www.wikidata.org/entity/Q41'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q9396')]
[('country2', 'http://www.wikidata.org/entity/Q41'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q28472762')]
[('country2', 'http://www.wikidata.org/entity/Q41'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q614920')]
[('country2', 'http://www.wikidata.org/entity/Q41'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q122141')]
[('country2', 'http://www.wikidata.org/entity/Q41'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q113215')]
[('country2', 'http://www.wikidata.org/entity/Q40'), ('Number', '1'), ('p', 'http://www.wikidata.org/entity/Q93256355')]
[('country2', 'http://www.wikidata.org

107

In [67]:
#What can we say about industry sectors in various countries
#here i return the number of industries for industry sector in various countries even outside the EU
queryString = """
SELECT  COUNT(?p) ?industry ?country
WHERE { 

{ 

?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {

?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}
?p wdt:P452 ?industry.
?p <http://schema.org/name> ?business.

} GROUP BY ?industry ?country
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2'), ('industry', 'http://www.wikidata.org/entity/Q474883'), ('country', 'http://www.wikidata.org/entity/Q801')]
[('callret-0', '16'), ('industry', 'http://www.wikidata.org/entity/Q29585175'), ('country', 'http://www.wikidata.org/entity/Q183')]
[('callret-0', '7'), ('industry', 'http://www.wikidata.org/entity/Q29585313'), ('country', 'http://www.wikidata.org/entity/Q183')]
[('callret-0', '6'), ('industry', 'http://www.wikidata.org/entity/Q29584705'), ('country', 'http://www.wikidata.org/entity/Q183')]
[('callret-0', '5'), ('industry', 'http://www.wikidata.org/entity/Q837171'), ('country', 'http://www.wikidata.org/entity/Q739')]
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q190117'), ('country', 'http://www.wikidata.org/entity/Q16957')]
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q216053'), ('country', 'http://www.wikidata.org/entity/Q29')]
[('callret-0', '2'), ('industry', 'http://www.wikidata.org/entity/Q3966'), ('coun

50

In [66]:
#What can we say about industry sectors in various countries
#here i return the number of industries for industry sector in various countries in the EU
queryString = """
SELECT  COUNT(?p) ?industry ?country
WHERE { 

{ 
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?country wdt:P463 wd:Q458.
?p wdt:P17 ?country  . 
?p wdt:P31 wd:Q4830453  . 
}
?p wdt:P452 ?industry.
?p <http://schema.org/name> ?business.

} GROUP BY ?industry ?country
 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q29585295'), ('country', 'http://www.wikidata.org/entity/Q145')]
[('callret-0', '2'), ('industry', 'http://www.wikidata.org/entity/Q3997851'), ('country', 'http://www.wikidata.org/entity/Q213')]
[('callret-0', '3'), ('industry', 'http://www.wikidata.org/entity/Q1961917'), ('country', 'http://www.wikidata.org/entity/Q183')]
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q46970'), ('country', 'http://www.wikidata.org/entity/Q183')]
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q1423777'), ('country', 'http://www.wikidata.org/entity/Q183')]
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q1625796'), ('country', 'http://www.wikidata.org/entity/Q213')]
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q130693'), ('country', 'http://www.wikidata.org/entity/Q183')]
[('callret-0', '1'), ('industry', 'http://www.wikidata.org/entity/Q899277'), ('count

200